# 🤖 워크플로우에서 에이전트 사용하기

> **📖 Learn 사이트 링크**  
> https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/agents-in-workflows?pivots=programming-language-python  
>
> **💻 원본 소스 코드**  
> https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/azure_ai_agents_streaming.py

## 📋 개요

이 과정에서는 Agent Framework를 사용하여 **AI 에이전트를 워크플로에 통합**하는 방법을 보여줍니다. 

콘텐츠 생성, 검토 및 기타 협업 작업을 위해 특화된 AI 에이전트의 기능을 활용하는 워크플로우를 만드는 방법을 다룹니다.

### 🎯 학습 목표

다음과 같은 워크플로우를 생성합니다:

| 순서 | 단계 | 설명 |
|------|------|------|
| 1️⃣ | **에이전트 생성** | Azure AI Agent Service를 사용하여 지능형 에이전트를 생성 |
| 2️⃣ | **Writer 에이전트** | 프롬프트에 따라 콘텐츠를 생성하는 에이전트 구현 ✍️ |
| 3️⃣ | **Reviewer 에이전트** | 콘텐츠에 대한 피드백을 제공하는 에이전트 구현 👀 |
| 4️⃣ | **워크플로우 연결** | 에이전트들을 순차적인 파이프라인으로 연결 🔗 |
| 5️⃣ | **실시간 스트리밍** | 에이전트들이 요청을 처리하는 동안 실시간 업데이트 스트리밍 📡 |

## 📦 1단계: 필수 종속성 가져오기

먼저 Azure AI 에이전트 및 워크플로에 필요한 구성 요소를 가져옵니다.

In [ ]:
from collections.abc import Awaitable, Callable
from contextlib import AsyncExitStack
from typing import Any

from agent_framework.azure import AzureAIAgentClient
from agent_framework import WorkflowBuilder, WorkflowOutputEvent, AgentRunUpdateEvent, ChatAgent
from azure.identity.aio import AzureCliCredential

## 🏭 2단계: Azure AI 에이전트 팩토리 생성하기

적절한 비동기 컨텍스트 처리를 통해 Azure AI 에이전트 생성을 관리하는 헬퍼 함수를 생성합니다.

In [ ]:
"""
샘플: 워크플로우 내 에이전트 스트리밍

Writer 에이전트가 콘텐츠를 생성한 후, Reviewer 에이전트가 이를 비평합니다.
워크플로우는 스트리밍을 사용하므로 각 에이전트가 토큰을 사용할 때마다 점진적인 AgentRunUpdateEvent 청크를 살펴볼 수 있습니다.

목적:
에이전트를 엣지로 직접 추가하여 채팅 에이전트를 WorkflowBuilder 파이프라인에 연결하는 방법을 보여줍니다.

시연:
- run_stream()을 사용할 경우, AgentRunUpdateEvent를 통해 에이전트 델타를 자동으로 스트리밍합니다.
- 에이전트가 워크플로우 모드로 구동합니다: run_stream()은 점진적인 업데이트를 내보내고, run()은 완전한 응답을 만들어 냅니다.
"""

async def create_azure_ai_agent() -> tuple[Callable[..., Awaitable[Any]], Callable[[], Awaitable[None]]]:
    """
    Azure AI agent 팩토리와 close 함수를 생성하는 헬퍼 메서드입니다. 🏭✨
    이는 비동기 컨텍스트 관리자가 적절히 처리되도록 보장합니다.
    """
    stack = AsyncExitStack()
    cred = await stack.enter_async_context(AzureCliCredential())

    client = await stack.enter_async_context(AzureAIAgentClient(credential=cred))

    async def agent(**kwargs: Any) -> Any:
        return await stack.enter_async_context(client.as_agent(**kwargs))

    async def close() -> None:
        await stack.aclose()

    return agent, close

## 🤖 3단계: Azure AI Agent 생성하기

컨텐츠 생성과 리뷰를 담당하는 **2개의 특화된 에이전트**를 생성합니다.

### 에이전트 역할

| 에이전트 | 역할 | 아이콘 |
|----------|------|--------|
| **Writer** | 훌륭한 콘텐츠 작성 및 편집 | ✍️📝 |
| **Reviewer** | 실행 가능한 피드백 제공 | 👀✅ |

In [ ]:
def create_writer_agent(client: AzureAIAgentClient) -> ChatAgent:
    return client.as_agent(
        name="Writer",
        instructions=(
            "당신은 훌륭한 콘텐츠 작성자입니다. "
            "새로운 콘텐츠를 만들고 피드백에 따라 콘텐츠를 편집합니다."
            "한국어로 콘텐츠를 제공해주세요. ✍️📝"
        ),
    )


def create_reviewer_agent(client: AzureAIAgentClient) -> ChatAgent:
    return client.as_agent(
        name="Reviewer",
        instructions=(
            "당신은 훌륭한 콘텐츠 리브어입니다. "
            "작성자에게 제공된 콘텐츠에 대한 실행 가능한 피드백을 제공하세요. "
            "가능한 한 간결하게 피드백을 제공하세요. "
            "한국어로 피드백을 제공해주세요. 👀✅"
        ),
    )


## 🔄 4단계: 워크플로우를 구성하고, 스트리밍을 실행합니다

빌더를 사용하여 에이전트를 **순차적인 워크플로우**에 연결하고, 에이전트들로부터 실시간 업데이트를 관찰하기 위해 스트리밍으로 워크플로우를 실행합니다.

In [ ]:
async def main() -> None:
    async with AzureCliCredential() as cred, AzureAIAgentClient(credential=cred) as client:
        # Build the workflow by adding agents directly as edges.
        # Agents adapt to workflow mode: run_stream() for incremental updates, run() for complete responses.
        workflow = (
            WorkflowBuilder()
            .register_agent(lambda: create_writer_agent(client), name="writer")
            .register_agent(lambda: create_reviewer_agent(client), name="reviewer", output_response=True)
            .set_start_executor("writer")
            .add_edge("writer", "reviewer")
            .build()
        )

        last_executor_id: str | None = None

        events = workflow.run_stream("가성비 있고 운전이 즐거운 신형 전기 SUV를 위한 슬로건을 한국어로 만드세요.")
        async for event in events:
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"{eid}:", end=" ", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("\n===== 최종 출력 =====")
                print(event.data)


if __name__ == "__main__":
    await main()

## 🔍 작동 방식

이 워크플로우는 다음과 같은 구조로 작동합니다:

### 🔧 구성 요소

| 구성 요소 | 설명 | 
|-----------|------|
|🔐 **Azure AI Client 설정** | 인증을 위해 Azure CLI 자격 증명과 함께 AzureAIAgentClient를 사용 |  
|🏭 **에이전트 팩토리 패턴** | 여러 에이전트에 대한 비동기 컨텍스트 수명 주기를 관리하는 팩토리 함수 |  
|➡️ **순차 처리** | Writer 에이전트가 먼저 콘텐츠를 생성한 후 Reviewer 에이전트로 전달 |  
|📡 **스트리밍 업데이트** | AgentRunUpdateEvent는 에이전트가 응답을 생성할 때 실시간 토큰 업데이트를 제공 |  
|🧹 **컨텍스트 관리** | AsyncExitStack을 사용하여 Azure AI 리소스를 적절하게 정리 |  

---

## 💡 주요 개념

### 🌟 핵심 기술

> **Azure AI Agent Service** :  고급 추론 기능을 갖춘 클라우드 기반 AI 에이전트  
> **AgentRunUpdateEvent** : 에이전트 실행 중 실시간 스트리밍 업데이트  
> **AsyncExitStack** : 여러 리소스에 대한 적절한 비동기 컨텍스트 관리  
> **에이전트 팩토리 패턴** :  공유 클라이언트 구성으로 재사용 가능한 에이전트 생성  
> **순차 워크플로우** :  에이전트가 파이프라인으로 연결되어 출력이 한 에이전트에서 다음 에이전트로 흐름  

---

## 🎯 워크플로우 흐름도

```
사용자 입력 → Writer 에이전트 ✍️ → Reviewer 에이전트 👀 → 최종 출력 ✅
                  (콘텐츠 생성)         (피드백 제공)
```